In [1]:
import time
import copy

import torch
import torchvision
import torchvision.transforms as T
import torch.nn as nn
from torch.utils.data import DataLoader, random_split

In [2]:
transformer = T.Compose([
        T.RandomHorizontalFlip(),
        T.Grayscale(3), 
        T.ToTensor(), 
        T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

In [3]:
train_data = torchvision.datasets.FashionMNIST('./data', transform=transformer)
train_data, validation_data = random_split(train_data, (50000, 10000))
test_data = torchvision.datasets.FashionMNIST('./data', transform=transformer, train=False)

In [4]:
data_train = DataLoader(train_data, batch_size=256, shuffle=True, num_workers=8)
data_val = DataLoader(validation_data, batch_size=256, shuffle=False, num_workers=8)
data_test = DataLoader(test_data, batch_size=256, shuffle=False, num_workers=8)

In [5]:
def train_model(model, criterion, optimizer, data_loader, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(1, num_epochs + 1):
        print(f'Epoch {epoch}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in data_loader[phase]:

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
#             if phase == 'train':
#                 scheduler.step()

            epoch_loss = running_loss / len(data_loader[phase].dataset)
            epoch_acc = running_corrects.double() / len(data_loader[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [6]:
model = torchvision.models.resnet18(pretrained=True)
# for param in model.parameters():
#     param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 10)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 7)

In [8]:
model = train_model(model, criterion, optimizer, {'train': data_train, 'val': data_val}, scheduler, 20)

Epoch 1/20
----------
train Loss: 0.7932 Acc: 0.7180
val Loss: 3.5548 Acc: 0.7295

Epoch 2/20
----------
train Loss: 0.4116 Acc: 0.8511
val Loss: 0.3558 Acc: 0.8680

Epoch 3/20
----------
train Loss: 0.3311 Acc: 0.8785
val Loss: 0.3143 Acc: 0.8904

Epoch 4/20
----------
train Loss: 0.2894 Acc: 0.8941
val Loss: 0.2855 Acc: 0.9006

Epoch 5/20
----------
train Loss: 0.2694 Acc: 0.9022
val Loss: 0.2718 Acc: 0.9018

Epoch 6/20
----------
train Loss: 0.2724 Acc: 0.8988
val Loss: 0.3024 Acc: 0.8921

Epoch 7/20
----------
train Loss: 0.2407 Acc: 0.9109
val Loss: 0.2855 Acc: 0.8996

Epoch 8/20
----------
train Loss: 0.2287 Acc: 0.9151
val Loss: 0.2648 Acc: 0.9077

Epoch 9/20
----------
train Loss: 0.2128 Acc: 0.9208
val Loss: 0.2735 Acc: 0.9046

Epoch 10/20
----------
train Loss: 0.2089 Acc: 0.9221
val Loss: 0.3098 Acc: 0.8902

Epoch 11/20
----------
train Loss: 0.2126 Acc: 0.9212
val Loss: 0.7374 Acc: 0.8131

Epoch 12/20
----------
train Loss: 0.4568 Acc: 0.8436
val Loss: 0.3471 Acc: 0.8725

E

In [9]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

In [10]:
check_accuracy(data_test, model)

Got 9053 / 10000 with accuracy 90.53
